In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # this suppresses a warning about pandas from tqdm
import os
import sys
import zipfile
import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import Layout
import webbrowser    # for opening web pages
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, clear_output
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

In [12]:
def do_something(change, pno): 
    if change.get("owner") == dropdown:
        with out: 
            print(pno)
    else:
        with out: 
            print("no")

In [13]:
dropdown = widgets.Dropdown(options = ['None', 'P123456'], value='None', description = 'Select Pno')
pno = "P020345"
out = widgets.Output()
dropdown.observe(do_something, 'value', pno)
col = widgets.VBox([dropdown, out])
display(col)